In [17]:
import os, sys
import numpy as np
from astropy.time import Time, TimeMJD
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy import units
from tqdm import tqdm_notebook
from astropy.io import fits
from matplotlib import gridspec
from scipy.interpolate import interp1d
from celluloid import Camera
from matplotlib.animation import FuncAnimation

parula = np.load('/Users/belugawhale/parula_colors.npy')

params = Table.read('/Users/belugawhale/Documents/GitHub/wasp39b_paper/scripts/rcParams.txt', format='csv')
for i, name in enumerate(params['name']):
    try:
        plt.rcParams[name] = float(params['value'][i])
    except:
        plt.rcParams[name] = params['value'][i]
plt.rcParams['font.size'] = 16
        
%load_ext autoreload
%autoreload 2
sys.path.append('/Users/belugawhale/Documents/GitHub/cos_flares/src')
from cos_reduction import *
from cos_flares import FlaresWithCOS

COLOR = 'k'
plt.rcParams['font.size'] = 20
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

plt.rcParams['xtick.major.width'] = 3
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['xtick.major.size']  = 10 #12
plt.rcParams['ytick.major.size']  = 10 #12

plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.minor.size']  = 6
plt.rcParams['ytick.minor.size']  = 6

plt.rcParams['axes.linewidth'] = 3

plt.rcParams['font.size'] = 20
plt.rcParams['text.color'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['axes.edgecolor'] = COLOR
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['legend.facecolor'] = 'none'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
predicted_Tc = np.array([2459571.90279, 2459596.65227, 
                         2459613.15229, 2459786.41361])
predicted_Tc = np.array(['2021-12-23 09:40', '2022-01-17 03:39',
                         '2022-02-02 15:39', '2022-07-25 21:55'])

predicted_ingress = np.array(['2021-12-23 07:13', '2022-01-17 01:12',
                              '2022-02-02 13:12', '2022-07-25 19:28'])

predicted_egress = np.array(['2021-12-23 12:06', '2022-01-17 06:06',
                             '2022-02-02 18:06', '2022-07-26 00:22'])

In [10]:
((Time(predicted_egress).mjd - Time(predicted_ingress).mjd)*units.day).to(units.hour)

<Quantity [4.88333333, 4.9       , 4.9       , 4.9       ] h>

In [11]:
flux = np.load('/Users/belugawhale/Documents/HST_COS/V1298_Tau/30s/reduced/flux.npy')
flux_err = np.load('/Users/belugawhale/Documents/HST_COS/V1298_Tau/30s/reduced/error.npy')
elements = np.load('/Users/belugawhale/Documents/HST_COS/V1298_Tau/30s/reduced/elements.npy')
times = np.load('/Users/belugawhale/Documents/HST_COS/V1298_Tau/30s/reduced/times.npy')

## Identifying flares in visit 3

In [12]:
inds = np.where(np.diff(times) > 10)[0]

visit = np.zeros(len(times))
inds = np.append([0], inds)
inds = np.append(inds, [len(times)])

for i in range(len(inds)-1):
    visit[inds[i]+1:inds[i+1]+1] = i

In [18]:
wavelength = np.load('/Users/belugawhale/Documents/HST_COS/V1298_Tau/30s/reduced/wavelength.npy')


fwc = FlaresWithCOS(wavelength=wavelength,
                    flux=flux,
                    flux_err=flux_err,
                    time=times,
                    orbit=visit)

fwc.load_line_table(path='/Users/belugawhale/Documents/GitHub/cos_flares/',
                    fname='line_table_V1298tau.txt')
fwc.load_lsf_model(fname='/Users/belugawhale/Documents/HST_COS/AUMic/aa_LSFTable_G130M_1222_LP4_cn.dat')

for ion in fwc.line_table['ion']:
    fwc.measure_ew(ion)

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(14,10))

gs0 = gridspec.GridSpec(2,1, figure=fig, height_ratios=[1.5,1])

gs00 = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs0[0], hspace=0.5)

ax1 = fig.add_subplot(gs00[0, 0])
ax2 = fig.add_subplot(gs00[1,0])
ax3 = fig.add_subplot(gs00[2,0])
ax4 = fig.add_subplot(gs00[3,0])

ax5 = fig.add_subplot(gs0[1,0])

fig.set_facecolor('w')
phase = np.full(len(times[visit==0]), np.nan)
carbon = np.full(len(times[visit==0]), np.nan)
factor=10**-13

for j, ax in enumerate([ax1,ax2,ax3,ax4]):
    q1 = visit == j+1
    
    ax.axvline(Time(predicted_Tc[j]).mjd-times[q1][0], color='k')

    ax.axvspan(Time(predicted_ingress[j]).mjd-times[q1][0], 
                Time(predicted_egress[j]).mjd-times[q1][0],
                color='k', alpha=0.1, lw=0)

    
    ax.plot(times[q1]-times[q1][0], fwc.width_table['CII_10'][q1]/factor, '.', c=parula[j*70])
    
    ax5.plot(times[q1] - Time(predicted_Tc[j]).mjd, fwc.width_table['CII_10'][q1]/factor, '.',
            c=parula[j*70])
    
    phase  = np.append(phase,  times[q1] - Time(predicted_Tc[j]).mjd)
    carbon = np.append(carbon, fwc.width_table['CII_10'][q1])
    
    ax.set_ylim(np.nanmin(fwc.width_table['CII_10'][q1]/factor),
                np.nanmax(fwc.width_table['CII_10'][q1]/factor))
    
    #ax.text(s='Visit {}'.format(j+1), x=-0.13, y=1, fontsize=16)
    
ax5.axvline(0, color='k')
Tdur = (4.9*units.hr).to(units.day).value
ax5.axvspan(0.0-Tdur/2.0, 0.0+Tdur/2.0, color='k', alpha=0.1, lw=0)

ax4.set_xlabel('t - $t_0$ [days]', fontsize=20)
ax5.set_xlabel('t - $T_0$ [days]', fontsize=20)

ax4.set_ylabel('Flux [10$^{-13}$ erg s$^{-1}$ cm$^{-2}$]', fontsize=20)
ax5.set_xticks(np.arange(-0.15, 0.4, 0.05))
ax5.set_xticklabels(np.round(np.arange(-0.15, 0.4, 0.05),2), fontsize=16)

plt.savefig('/Users/belugawhale/Desktop/hst_v1298tau.png', dpi=200, rasterize=True,
            bbox_inches='tight')

In [ ]:
%matplotlib inline

orbits = np.zeros(len(times[visit==0]))
transit = np.zeros(len(orbits))

for i in range(1,5):
    q = (visit == i) 
    o = np.zeros(len(times[q]))
    t = np.zeros(len(times[q]))
    
    seps = np.where(np.diff(times[q]) > 0.01)[0] + 1
    seps = np.append([0], seps)
    seps = np.append(seps, len(times[q]))
    
    t_i = Time(predicted_ingress[i-1]).mjd
    t_e = Time(predicted_egress[i-1]).mjd
    time_chunk = times[q]
    
    
    for j in range(len(seps)-1):
        o[seps[j]:seps[j+1]] += j 
        
        if (i==3 and j < 3) or (i==4 and j<2):
            t[seps[j]:seps[j+1]] = 2
        
    tt = np.where((times[q] >= t_i) & (times[q] <= t_e))[0]
    t[tt] = 1
    #t[seps[j]:seps[j+1]] = 1
    
    plt.scatter(np.arange(0,len(times[q]),1), times[q], c=t, vmin=0, vmax=2)

    #plt.show()
    plt.close()
    orbits = np.append(orbits, o)
    transit = np.append(transit, t)

In [ ]:
fwc.load_line_table(path='/Users/belugawhale/Documents/GitHub/cos_flares/',
                    fname='line_table_V1298tau.txt')

for ion in fwc.line_table['ion']:
    fwc.measure_ew(ion)

In [ ]:
v = 4
q_oot = (visit == v) & (transit == 0)
med_oot = np.nanmean(flux[q_oot], axis=0)

q_it = (visit == v) & (transit == 1)
med_it = np.nanmean(flux[q_it], axis=0)

%matplotlib notebook
plt.plot(wavelength[q_oot][0], med_oot)
plt.plot(wavelength[q_it][0], med_it)
plt.yscale('log')

for i in range(len(fwc.line_table)):
    plt.axvline(fwc.line_table['wave_c'][i], color='k', zorder=0, lw=1)
    plt.text(s=fwc.line_table['ion'][i], x=fwc.line_table['wave_c'][i], y=10**-14, color='k',
             rotation=90, fontsize=16)

#plt.xlim(1300,1309)

In [ ]:
ion = 'NV_2'

COLOR = 'k'
plt.rcParams['font.size'] = 20

%matplotlib inline
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(28,14), sharex=True)
axes = axes.reshape(-1)
fig.set_facecolor('w')

factor = 10**-13

for n, ion in enumerate(['NV_1', 'NV_2', 'CII_10', 'CII_11', 'SiII', 'SiIII_1',
                         'SiIV', 'SiIV_2']):

    for v in range(2,5):

        q_oot = (visit == v) & (transit == 0)

        q_it = (visit == v) & (transit == 1)

        bins, binned, binned_err = np.array([]), np.array([]), np.array([])

        for o in range(5):
            q = (visit==v) & (orbits==o) & (transit < 2)
            temp_bins = np.linspace(phase[q][0], phase[q][-1], 6)
            bins = np.append(bins, (temp_bins[1:]+temp_bins[:-1])/2.0)

            for i in range(len(temp_bins)-1):
                ind = np.where((phase[q]>=temp_bins[i]) & (phase[q]<temp_bins[i+1]))[0]

                binned = np.append(binned, 
                                   np.nanmean(fwc.width_table[ion][q][ind]))

                binned_err = np.append(binned_err,
                                       np.nanmedian(fwc.error_table[ion][q][ind])/len(ind))

        axes[n].errorbar(phase[q_oot], fwc.width_table[ion][q_oot]/factor, #yerr=fwc.error_table[ion][q_oot],
                 marker='o', linestyle='',
                 alpha=0.3, color='deepskyblue',
                 markeredgewidth=0, lw=0)

        axes[n].errorbar(phase[q_it], fwc.width_table[ion][q_it]/factor, #yerr=fwc.error_table[ion][q_it],
                     marker='o', linestyle='',
                     alpha=0.3, color='darkorange',
                     markeredgewidth=0, lw=0)

        axes[n].errorbar(bins, binned/factor, marker='o', 
                     markeredgecolor='k', linestyle='', zorder=10, 
                     markerfacecolor='w')
        axes[n].errorbar(bins, binned/factor, yerr=binned_err/factor, 
                         marker='o', color='k', linestyle='', zorder=4,
                         alpha=0.75)

        axes[n].set_xticks(np.arange(-0.05,0.35,0.05))
        

        yticks = np.linspace(np.nanmin(fwc.width_table[ion]/factor),
                             np.nanmax(fwc.width_table[ion]/factor), 5)
        
        if 'SiIV' in ion:
            yticks = np.linspace(-0.1, 8, 5)
        if 'SiIV_2' in ion:
            yticks = np.linspace(-0.1, 6, 5)
        axes[n].set_ylim(yticks[0], yticks[-1])
        axes[n].set_yticks(yticks)
        axes[n].set_yticklabels(np.round(yticks,2), fontsize=18)
        
        title = r'$\lambda$ = ' + str(np.round(fwc.line_table[fwc.line_table['ion']==ion]['wave_c'][0], 2))
        axes[n].set_title(title + ' ({})'.format(ion.split('_')[0]), fontsize=20)
        axes[n].axvline(0, color='darkorange', zorder=1)
        

for n in range(len(axes)-2, len(axes)):
    axes[n].set_xlabel('T - T$_0$ [days]', fontsize=26)
    axes[n].set_xticks(np.arange(-0.05,0.35,0.05))
    axes[n].set_xticklabels(np.round(np.arange(-0.05,0.35,0.05),2), fontsize=20)

axes[2].set_ylabel('Flux [10$^{-13}$ erg s$^{-1}$ cm$^{-2}$]', fontsize=26)

plt.savefig('/Users/belugawhale/Desktop/transit_lc.png', dpi=250,
            rasterize=True, bbox_inches='tight')

## combined multiplets

In [ ]:
ion = 'NV_2'

COLOR = 'k'
plt.rcParams['font.size'] = 20

%matplotlib inline
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(28,10), sharex=True)
axes = axes.reshape(-1)
fig.set_facecolor('w')

factor = 10**-13

for n, ion in enumerate(['SiIII_1', 'SiIV_combined', 'CII_combined', 'NV_combined']):#, 'SiIII_3']):

    for v in range(3,5):

        q_oot = (visit == v) & (transit == 0)

        q_it = (visit == v) & (transit == 1)
        
        norm_reg = (visit == v) & (transit == 0) & (phase >= 0.2) & (phase <= 0.25)
        normalization = np.nanmean(fwc.width_table[ion][norm_reg])

        bins, binned, binned_err = np.array([]), np.array([]), np.array([])

        for o in range(5):
            q = (visit==v) & (orbits==o) & (transit < 2)
            temp_bins = np.linspace(phase[q][0], phase[q][-1], 6)
            bins = np.append(bins, (temp_bins[1:]+temp_bins[:-1])/2.0)

            for i in range(len(temp_bins)-1):
                ind = np.where((phase[q]>=temp_bins[i]) & (phase[q]<temp_bins[i+1]))[0]

                binned = np.append(binned, 
                                   np.nanmean(fwc.width_table[ion][q][ind]))

                binned_err = np.append(binned_err,
                                       np.nanmedian(fwc.error_table[ion][q][ind])/len(ind))

        axes[n].errorbar(phase[q_oot], fwc.width_table[ion][q_oot]/normalization, #yerr=fwc.error_table[ion][q_oot],
                 marker='o', linestyle='',
                 alpha=0.3, color='deepskyblue',
                 markeredgewidth=0, lw=0)

        axes[n].errorbar(phase[q_it], fwc.width_table[ion][q_it]/normalization, #yerr=fwc.error_table[ion][q_it],
                     marker='o', linestyle='',
                     alpha=0.3, color='darkorange',
                     markeredgewidth=0, lw=0)

        axes[n].errorbar(bins, binned/normalization, marker='o', 
                     markeredgecolor='k', linestyle='', zorder=10, 
                     markerfacecolor='w')
        axes[n].errorbar(bins, binned/normalization, yerr=binned_err/normalization, 
                         marker='o', color='k', linestyle='', zorder=4,
                         alpha=0.75)

        axes[n].set_xticks(np.arange(-0.05,0.35,0.05))
        

        yticks = np.linspace(np.nanmin(fwc.width_table[ion]/normalization),
                             np.nanmax(fwc.width_table[ion]/normalization), 5)
        
        if 'SiIV' in ion:
            yticks = np.linspace(-0.1, 3, 5)
        else:
            yticks = np.linspace(-0.1, 2, 5)
        axes[n].set_ylim(yticks[0], yticks[-1])
        axes[n].set_yticks(yticks)
        axes[n].set_yticklabels(np.round(yticks,2), fontsize=18)
        
        title = r'$\lambda$ = ' + str(np.round(fwc.line_table[fwc.line_table['ion']==ion]['wave_c'][0], 2))
        axes[n].set_title(title + ' ({})'.format(ion.split('_')[0]), fontsize=20)
        axes[n].axvline(0, color='k', zorder=1, alpha=0.3)
        axes[n].axhline(1, color='k', zorder=1, alpha=0.3)
        

for n in range(len(axes)-1, len(axes)):
    axes[n].set_xlabel('T - T$_0$ [days]', fontsize=26)
    axes[n].set_xticks(np.arange(-0.05,0.35,0.05))
    axes[n].set_xticklabels(np.round(np.arange(-0.05,0.35,0.05),2), fontsize=20)

axes[2].set_ylabel('Flux [10$^{-13}$ erg s$^{-1}$ cm$^{-2}$]', fontsize=26)

#plt.savefig('/Users/belugawhale/Desktop/transit_lc.png', dpi=250,
#            rasterize=True, bbox_inches='tight')

In [ ]:
ion = 'CII_combined'

b = 40
vel_bins = np.arange(fwc.line_table[fwc.line_table['ion']==ion]['vmin'], 
                     fwc.line_table[fwc.line_table['ion']==ion]['vmax']+b, b)

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20,10), sharex=True)
axes = axes.reshape(4,3)
axes = axes.reshape(-1)
fig.set_facecolor('w')

x = 0
for v in range(2,6):
    
    if v == 5:
        q_oot = (visit > 1) & (transit == 0) & (visit < 5)
        med_oot = np.nanmean(flux[q_oot], axis=0)

        q_it = (visit >1) & (transit == 1) & (visit < 5) #& (phase < 0.1)
        med_it = np.nanmean(flux[q_it], axis=0)


        velocity, _ = fwc.to_velocity(wavelength[q_oot][100], 
                               mid=fwc.line_table[fwc.line_table['ion']==ion]['wave_c'])

        q_w = ( (velocity.value >= fwc.line_table[fwc.line_table['ion']==ion]['vmin']) & 
                (velocity.value <= fwc.line_table[fwc.line_table['ion']==ion]['vmax']) )

        
    else:
        q_oot = (visit == v) & (transit == 0) 
        med_oot = np.nanmean(flux[q_oot], axis=0)

        q_it = (visit == v) & (transit == 1) #& (phase < 0.1)
        med_it = np.nanmean(flux[q_it], axis=0)


        velocity, _ = fwc.to_velocity(wavelength[q_oot][0], 
                               mid=fwc.line_table[fwc.line_table['ion']==ion]['wave_c'])

        q_w = ( (velocity.value >= fwc.line_table[fwc.line_table['ion']==ion]['vmin']) & 
                (velocity.value <= fwc.line_table[fwc.line_table['ion']==ion]['vmax']) )

    axes[x].plot(velocity[q_w], med_oot[q_w], 'r')
    axes[x].plot(velocity[q_w], med_it[q_w], 'k')

    alpha = 0.2
    axes[x+4].plot(velocity[q_w], med_it[q_w] - med_oot[q_w], 'k', alpha=alpha)

    axes[x+8].plot(velocity[q_w], med_it[q_w]/med_oot[q_w], 'k', alpha=alpha)
    
    for i in range(len(vel_bins)-1):
        ebar_dict = {'marker':'o', 'markerfacecolor':'w',
                     'zorder':10, 'color':'k', 'ms':8, 'lw':1.5,
                     'markeredgewidth':1.5}
        
        inds = ( (velocity.value >= vel_bins[i]) & (velocity.value <= vel_bins[i+1]))
        axes[x+4].errorbar((vel_bins[i]+vel_bins[i+1])/2.0, 
                           np.nanmean(med_it[inds] - med_oot[inds]),
                           yerr=np.nanstd(med_it[inds] - med_oot[inds]),
                           **ebar_dict)
        
        axes[x+8].errorbar((vel_bins[i]+vel_bins[i+1])/2.0, 
                           np.nanmean(med_it[inds] / med_oot[inds]),
                           yerr=np.nanstd(med_it[inds] / med_oot[inds]),
                           **ebar_dict)

    for i,ax in enumerate([axes[x+4], axes[x+8]]):
        ax.axhline(i, color='darkorange')
        
    axes[x+8].set_ylim(0,2)
    axes[x+8].set_xlim(velocity[q_w].value.min(), velocity[q_w].value.max())
    axes[x+4].set_xlim(velocity[q_w].value.min(), velocity[q_w].value.max())

    #plt.yscale('log')
    x += 1
    
fs = 16
axes[0].set_ylabel('Flux Density', fontsize=fs)
axes[4].set_ylabel('Difference (black-red)', fontsize=fs)
axes[8].set_ylabel('Ratio (black/red)', fontsize=fs)

plt.xlim(-200,320)

plt.show()

## Co-adding lines in the same velocity space (Linsky et al 2010)

In [ ]:
mean1 = np.nanmean(fwc.flux[q], axis=0)


In [ ]:
fwc.line_table[fwc.line_table['ion']=='CII_10']

In [ ]:
v = 2

ions = ['CII_10', 'CII_11']

factor = 1e14
b = 100
vel_bins = np.linspace(-200, 200, b)

q_oot = (visit == v) & (transit == 0)
q_it  = (visit == v) & (transit == 1)

comb, temp_flux, temp_err = [], [], []

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18,10),
                         sharex=True)
axes = axes.reshape(-1)

name = ions[0].split('_')[0]
axes[0].set_title('{0} ({1} $\AA$)'.format(name, 
                                          np.round(fwc.line_table[fwc.line_table['ion']==ions[0]]['wave_c'][0], 2)))
axes[1].set_title('{0} ({1} $\AA$)'.format(name,
                                          np.round(fwc.line_table[fwc.line_table['ion']==ions[1]]['wave_c'][0], 2)))
axes[2].set_title('{0} (Combined)'.format(name))

x = 0
color = ['r', 'k']
for i, q in enumerate([q_oot, q_it]):
    
    mean = np.nanmean(fwc.flux[q], axis=0)
    err = np.sqrt(np.nansum(fwc.flux_err[q]**2, axis=0))/len(fwc.flux)

    vel1, _ = fwc.to_velocity(wavelength[q][100], 
                              mid=fwc.line_table[fwc.line_table['ion']==ions[0]]['wave_c'])

    vel2, _ = fwc.to_velocity(wavelength[q][100], 
                              mid=fwc.line_table[fwc.line_table['ion']==ions[1]]['wave_c'])
    
    interp1, interp2 = np.array([]), np.array([])
    err1, err2 = np.array([]), np.array([])
    
    for j in range(len(vel_bins)-1):
        q1 = ((vel1.value >= vel_bins[j]) & 
              (vel1.value < vel_bins[j+1]))
        
        q2 = ((vel2.value >= vel_bins[j]) & 
              (vel2.value < vel_bins[j+1]))
        
        interp1 = np.append(interp1, np.nanmean(mean[q1]))
        interp2 = np.append(interp2, np.nanmean(mean[q2]))
        
        err1 = np.append(err1, np.nanmedian(err[q1]))
        err2 = np.append(err2, np.nanmedian(err[q2]))


    comb.append(interp1+interp2)
    temp_flux.append([interp1, interp2])
    temp_err.append([err1, err2])

x = (vel_bins[1:] + vel_bins[:-1])/2.0

for i in range(2):
    # line profile multiplet 1
    axes[0].plot(x, temp_flux[i][0]*factor, color[i])
    axes[0].fill_between(x, (temp_flux[i][0]-temp_err[i][0])*factor,
                         (temp_flux[i][0]+temp_err[i][0])*factor, 
                         alpha=0.2,
                         color=color[i], lw=0)
    
    # line profile multiplet 2
    axes[1].plot(x, temp_flux[i][1]*factor, color[i])
    axes[1].fill_between(x, (temp_flux[i][1]-temp_err[i][1])*factor,
                         (temp_flux[i][1]+temp_err[i][1])*factor, 
                         alpha=0.2,
                         color=color[i], lw=0)
    
    axes[2].plot(x, comb[i]*factor, color[i])
    
    # subtraction between in- vs out-of-transit line profile
    sub = temp_flux[1][i] - temp_flux[0][i]
    sub_e = np.sqrt(temp_err[1][i]**2.0 + temp_err[0][i]**2.0)
    axes[i+3].plot(x, sub*factor, 'k')
    axes[i+3].fill_between(x, (sub-sub_e)*factor,
                           (sub+sub_e)*factor,
                           alpha=0.2, color='k', lw=0)
    
    # ratio of in- vs out-of-transit line profile
    div = temp_flux[1][i] / temp_flux[0][i]
    div_e = np.sqrt((temp_err[1][i] / temp_flux[1][i])**2.0 +
                    (temp_err[0][i] / temp_flux[0][i])**2.0 )
    axes[i+6].plot(x, div, 'k')
    axes[i+6].fill_between(x, div-div_e,
                           div+div_e,
                           alpha=0.2, color='k', lw=0)

axes[5].plot(x, (comb[1] - comb[0])*factor, 'k')
axes[8].plot(x, comb[1] / comb[0], 'k')

for i in range(3,6):
    axes[i].axhline(0, color='darkorange', lw=2)

for i in range(6,9):
    axes[i].set_ylim(0.5,2.6)
    axes[i].axhline(1, color='darkorange', lw=2)
    axes[i].set_xlabel('Velocity [km s$^{-1}$]')

#plt.plot(varr, interp2(varr))
ax1.set_ylim(9*10**-16, 5*10**-14)
ax1.set_yscale('log')
plt.xlim(-130,170)

axes[0].set_ylabel('Flux Density\n[10$^{-14}$ erg s$^{-1}$ cm$^{-1} \AA^{-1}$]')
axes[3].set_ylabel('Difference\n(black-red)')
axes[6].set_ylabel('Ratio\n(black/red)');

In [ ]:
div

In [ ]:
ion = 'CII_11'

b = 40
vel_bins = np.arange(fwc.line_table[fwc.line_table['ion']==ion]['vmin'], 
                     fwc.line_table[fwc.line_table['ion']==ion]['vmax']+b, b)

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20,10), sharex=True)
axes = axes.reshape(4,3)
axes = axes.reshape(-1)
fig.set_facecolor('w')

x = 0
for v in range(2,6):
    
    if v == 5:
        q_oot = (visit > 1) & (transit == 0) & (visit < 5)
        med_oot = np.nanmean(flux[q_oot], axis=0)

        q_it = (visit >1) & (transit == 1) & (visit < 5) #& (phase < 0.1)
        med_it = np.nanmean(flux[q_it], axis=0)


        velocity, _ = fwc.to_velocity(wavelength[q_oot][100], 
                               mid=fwc.line_table[fwc.line_table['ion']==ion]['wave_c'])

        q_w = ( (velocity.value >= fwc.line_table[fwc.line_table['ion']==ion]['vmin']) & 
                (velocity.value <= fwc.line_table[fwc.line_table['ion']==ion]['vmax']+20) )

        
    else:
        q_oot = (visit == v) & (transit == 0) 
        med_oot = np.nanmean(flux[q_oot], axis=0)

        q_it = (visit == v) & (transit == 1) #& (phase < 0.1)
        med_it = np.nanmean(flux[q_it], axis=0)


        velocity, _ = fwc.to_velocity(wavelength[q_oot][0], 
                               mid=fwc.line_table[fwc.line_table['ion']==ion]['wave_c'])

        q_w = ( (velocity.value >= fwc.line_table[fwc.line_table['ion']==ion]['vmin']) & 
                (velocity.value <= fwc.line_table[fwc.line_table['ion']==ion]['vmax']+20) )

    axes[x].plot(velocity[q_w], med_oot[q_w], 'r')
    axes[x].plot(velocity[q_w], med_it[q_w], 'k')

    alpha = 0.2
    axes[x+4].plot(velocity[q_w], med_it[q_w] - med_oot[q_w], 'k', alpha=alpha)

    axes[x+8].plot(velocity[q_w], med_it[q_w]/med_oot[q_w], 'k', alpha=alpha)
    
    for i in range(len(vel_bins)-1):
        ebar_dict = {'marker':'o', 'markerfacecolor':'w',
                     'zorder':10, 'color':'k', 'ms':8, 'lw':1.5,
                     'markeredgewidth':1.5}
        
        inds = ( (velocity.value >= vel_bins[i]) & (velocity.value <= vel_bins[i+1]))
        axes[x+4].errorbar((vel_bins[i]+vel_bins[i+1])/2.0, 
                           np.nanmean(med_it[inds] - med_oot[inds]),
                           yerr=np.nanstd(med_it[inds] - med_oot[inds]),
                           **ebar_dict)
        
        axes[x+8].errorbar((vel_bins[i]+vel_bins[i+1])/2.0, 
                           np.nanmean(med_it[inds] / med_oot[inds]),
                           yerr=np.nanstd(med_it[inds] / med_oot[inds]),
                           **ebar_dict)

    for i,ax in enumerate([axes[x+4], axes[x+8]]):
        ax.axhline(i, color='darkorange')
        
    axes[x+8].set_ylim(0,2)
    axes[x+8].set_xlim(velocity[q_w].value.min(), velocity[q_w].value.max())
    axes[x+4].set_xlim(velocity[q_w].value.min(), velocity[q_w].value.max())

    #plt.yscale('log')
    x += 1
    
fs = 16
axes[0].set_ylabel('Flux Density', fontsize=fs)
axes[4].set_ylabel('Difference (black-red)', fontsize=fs)
axes[8].set_ylabel('Ratio (black/red)', fontsize=fs)


plt.show()

## maybe flare thing?

In [ ]:
ion = 'NV_2'

COLOR = 'k'
plt.rcParams['font.size'] = 20

%matplotlib inline
fig, axes = plt.subplots(figsize=(10,4), sharex=True)
axes = [axes]
fig.set_facecolor('w')

factor = 10**-13

for n, ion in enumerate(['CIII']):

    for v in range(2,5):

        q_oot = (visit == v) & (transit == 0)

        q_it = (visit == v) & (transit == 1)

        bins, binned, binned_err = np.array([]), np.array([]), np.array([])

        for o in range(5):
            q = (visit==v) & (orbits==o) & (transit < 2)
            temp_bins = np.linspace(phase[q][0], phase[q][-1], 6)
            bins = np.append(bins, (temp_bins[1:]+temp_bins[:-1])/2.0)

            for i in range(len(temp_bins)-1):
                ind = np.where((phase[q]>=temp_bins[i]) & (phase[q]<temp_bins[i+1]))[0]

                binned = np.append(binned, 
                                   np.nanmean(fwc.width_table[ion][q][ind]))

                binned_err = np.append(binned_err,
                                       np.nanmedian(fwc.error_table[ion][q][ind])/len(ind))

        axes[n].errorbar(phase[q_oot], fwc.width_table[ion][q_oot]/factor, #yerr=fwc.error_table[ion][q_oot],
                 marker='o', linestyle='',
                 alpha=0.7, color=parula[v*50],
                 markeredgewidth=0, lw=0)

        axes[n].errorbar(phase[q_it], fwc.width_table[ion][q_it]/factor, #yerr=fwc.error_table[ion][q_it],
                     marker='o', linestyle='',
                     alpha=0.7, color='darkorange',
                     markeredgewidth=0, lw=0)

        axes[n].errorbar(bins, binned/factor, marker='o', 
                     markeredgecolor='k', linestyle='', zorder=10, 
                     markerfacecolor='w')
        axes[n].errorbar(bins, binned/factor, yerr=binned_err/factor, 
                         marker='o', color='k', linestyle='', zorder=4,
                         alpha=0.75)

        axes[n].set_xticks(np.arange(-0.05,0.35,0.05))
        

        yticks = np.linspace(-0.15, 40, 5)
        axes[n].set_ylim(yticks[0], yticks[-1])
        axes[n].set_yticks(yticks)
        axes[n].set_yticklabels(np.round(yticks,2), fontsize=18)
        
        title = r'$\lambda$ = ' + str(np.round(fwc.line_table[fwc.line_table['ion']==ion]['wave_c'][0], 2))
        axes[n].set_title(title + ' ({})'.format(ion.split('_')[0]), fontsize=20)
        axes[n].axvline(0, color='darkorange', zorder=1)
        

for n in range(1):
    axes[n].set_xlabel('T - T$_0$ [days]', fontsize=26)
    axes[n].set_xticks(np.arange(0.15,0.175,0.01))
    axes[n].set_xticklabels(np.round(np.arange(0.15,0.175,0.01),2), fontsize=20)

axes[0].set_ylabel('Flux [10$^{-13}$ erg s$^{-1}$ cm$^{-2}$]', fontsize=20)
plt.xlim(0.15,0.175)

plt.savefig('/Users/belugawhale/Desktop/transit_lc_carbon.png', dpi=250,
            rasterize=True, bbox_inches='tight')

In [ ]:
meds = []
bins = np.array([])
diff = np.array([])
binned_visit = np.array([])

for v in range(2,5):

    for o in range(5):
        q = (visit==v) & (orbits==o) & (transit < 2)
        temp_bins = np.linspace(phase[q][0], phase[q][-1], 10)
        bins = np.append(bins, (temp_bins[1:]+temp_bins[:-1])/2.0)
        diff = np.append(diff, np.diff(temp_bins))#np.append(np.diff(temp_bins), np.diff(temp_bins)[0]))
        
        binned_visit = np.append(binned_visit, np.full(len(temp_bins)-1, v))

        for i in range(len(temp_bins)-1):
            ind = np.where((phase[q]>=temp_bins[i]) & (phase[q]<temp_bins[i+1]))[0]

            meds.append(np.nanmean(flux[q][ind], axis=0))
            
            
meds = np.array(meds)

In [ ]:
COLOR = 'k'
plt.rcParams['font.size'] = 20

ion = 'NV_combined'

v = 3
q_oot = (visit == v) & (transit == 0)
med_oot = np.nanmean(flux[q_oot], axis=0)

%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,4), sharex=True)
fig.set_facecolor('w')
camera = Camera(fig)

velocity, _ = fwc.to_velocity(wavelength[q_oot][0], mid=fwc.line_table[fwc.line_table['ion']==ion]['wave_c'])
velocity = velocity.value

ax.set_ylabel('Flux Density\n[10$^{-13}$ erg $\AA^{-1}$ s$^{-1}$ cm$^{-2}$]', fontsize=18)
ax.set_xlabel('Velocity [km s$^{-1}$]', fontsize=18)

plt.xlim(fwc.line_table[fwc.line_table['ion']==ion]['vmin'], 
         fwc.line_table[fwc.line_table['ion']==ion]['vmax'])

plt.ylim(0,2.75)
plt.yticks(np.arange(0,2.75,0.5), fontsize=16)

factor = 10**-14

argsort = np.argsort(bins)

for i in range(len(meds[bins<0.1])):
    
    v = int(binned_visit[argsort[i]])
    q_oot = (visit == v) & (transit == 0)
    med_oot = np.nanmean(flux[q_oot], axis=0)
    
    velocity, _ = fwc.to_velocity(wavelength[q_oot][0], mid=fwc.line_table[fwc.line_table['ion']==ion]['wave_c'])
    velocity = velocity.value
    
    #if v == 4:
    #    velocity = 150

    
    plt.plot(velocity, med_oot/factor, c='k', alpha=0.8, zorder=10)
    plt.plot(velocity, meds[argsort[i]]/factor, c=parula[int((binned_visit[argsort[i]]-2)*80+40)])
    plt.text(s='Phase = {0} to {1}'.format(np.round(bins[argsort[i]] - diff[argsort[i]], 3),
                                          np.round(bins[argsort[i]] + diff[argsort[i]], 3)),
             x=0, y=2.5, fontsize=16)
    camera.snap()


animation = camera.animate()

args = {'interval':10000}
animation.save('/Users/belugawhale/Desktop/{}.gif'.format(ion), *args)